In [22]:
import requests
from bs4 import BeautifulSoup
import pandas


In [16]:
params={}
params["ww_x_GPS"]=-1
params["ww_i_reportModel"]=133685247
params["ww_i_reportModelXsl"]=[133685270]
params["ww_x_UNITE_ACAD"]=249847
params["ww_x_PERIODE_ACAD"]=978181
params["ww_x_PERIODE_PEDAGO"]=2230106
params["ww_x_HIVERETE"]=2936286

In [17]:
r=requests.get("http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?",params)

In [18]:
print(r.url)

http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_UNITE_ACAD=249847&ww_x_PERIODE_PEDAGO=2230106&ww_x_HIVERETE=2936286&ww_i_reportModelXsl=133685270&ww_i_reportModel=133685247&ww_x_PERIODE_ACAD=978181&ww_x_GPS=-1


In [40]:
pandas.read_html(r.url,header=0,skiprows=1)[0].dropna(1)

,Civilité,Nom Prénom,Statut,No Sciper
0,Monsieur,Aeberhard François-Xavier,Présent,153066
1,Madame,Agarwal Megha,Présent,180027
2,Monsieur,Anagnostaras David,Présent,152232
3,Monsieur,Auroux Damien,Présent,177395
4,Monsieur,Awalebo Joseph,Présent,161970
5,Monsieur,Balet Ken,Présent,166258
6,Monsieur,Barazzutti Raphaël Pierre,Présent,173600
7,Monsieur,Bayramoglu Ersoy,Présent,178879
8,Madame,Benabdallah Zeineb,Présent,154573
9,Monsieur,Bettex Marc,Présent,160492


In [21]:
soup=BeautifulSoup(r.text,'lxml')

In [133]:
soup.find_all("tr")

[<tr><th colspan="10">
     Ces listes d'étudiants ne seront définitives qu'à la mi-août une fois la commission d'admission bachelor et la conférence d'examen passées afin de s'assurer que tous les étudiants nouvellement admis à l'EPFL y figurent et que les étudiants en situation d'échec définitif soient rayés de ces listes
    </th></tr>,
 <tr><th colspan="12"><font color="black">Echange IN, 2007-2008, Semestre automne</font>
  (17 ét.)
     </th></tr>,
 <tr><th>Civilité</th><th>Nom Prénom</th><th>Orientation Bachelor</th><th>Orientation Master</th><th>Spécialisation</th><th>Filière opt.</th><th>Mineur</th><th>Statut</th><th>Type Echange</th><th>Ecole Echange</th><th>No Sciper</th></tr>,
 <tr><td style="white-space:nowrap">Monsieur</td><td style="white-space:nowrap">Bouaziz Sofien</td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space:nowrap"></td><td style="white-space: